In [71]:
"""
Created on Wed Oct 19 15:48:00 2018

@author: Daniel Cuesta, Alejandro Garo
"""

import string, random
import numpy as np
import matplotlib.pyplot as plt
import sys
import time
import math
import cmath

from cmath import pi, exp

In [122]:
t =np.array([1,2,1,0])

In [132]:
def fft(t):
    
    k = len(t)
    
    if k == 1 :
        return t
    
    pow2 = math.ceil(math.log(k,2))
    print(pow2)
    
    t = np.array(list(t) + (2**pow2 - k) * [float(0)], dtype = complex)
    # t.append(np.zeros(2**log2K-k))
    
    print(t)
        
    c_e = t[::2]
    c_o = t[1::2]
    
    f_e = fft(c_e)
    f_o = fft(c_o)
    print("f_e ",f_e)
    print("f_o ",f_o)
    for i in range((2**pow2)-1):
        t[i] = f_e[i % 2**(pow2-1)] + exp(2j*pi*(i/2**pow2)) * f_o[i % 2**(pow2-1)]
    return t

In [ ]:
def invert_fft(t, fft_func=fft):
    pass

In [135]:
print(fft(fft(t)))
print (np.fft.fft(fft(t)))


2
[ 1.+0.j  2.+0.j  1.+0.j  0.+0.j]
1
[ 1.+0.j  1.+0.j]
f_e  [ 1.+0.j]
f_o  [ 1.+0.j]
1
[ 2.+0.j  0.+0.j]
f_e  [ 2.+0.j]
f_o  [ 0.+0.j]
f_e  [ 2.+0.j  1.+0.j]
f_o  [ 2.+0.j  0.+0.j]
2
[ 4. +0.00000000e+00j  1. +0.00000000e+00j  0. +2.44929360e-16j
  0. +0.00000000e+00j]
1
[ 4. +0.00000000e+00j  0. +2.44929360e-16j]
f_e  [ 4.+0.j]
f_o  [ 0. +2.44929360e-16j]
1
[ 1.+0.j  0.+0.j]
f_e  [ 1.+0.j]
f_o  [ 0.+0.j]
f_e  [ 4. +2.44929360e-16j  0. +2.44929360e-16j]
f_o  [ 1.+0.j  0.+0.j]
[ 5. +2.44929360e-16j  0. +2.44929360e-16j  3. +3.67394040e-16j
  0. +0.00000000e+00j]
2
[ 1.+0.j  2.+0.j  1.+0.j  0.+0.j]
1
[ 1.+0.j  1.+0.j]
f_e  [ 1.+0.j]
f_o  [ 1.+0.j]
1
[ 2.+0.j  0.+0.j]
f_e  [ 2.+0.j]
f_o  [ 0.+0.j]
f_e  [ 2.+0.j  1.+0.j]
f_o  [ 2.+0.j  0.+0.j]
[ 5. +2.44929360e-16j  4. -1.00000000e+00j  3. +2.44929360e-16j
  4. +1.00000000e+00j]
